In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
import seaborn as sns
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_log_error, mean_squared_error

rng = np.random.default_rng(2023)

In [2]:
from sklearn.metrics import mean_squared_log_error, mean_squared_error

---

In [3]:
df = pd.read_csv("https://raw.githubusercontent.com/avehtari/ROS-Examples/master/Earnings/data/earnings.csv").dropna().drop(columns=["earnk"])

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.smokenow.unique()

In [8]:
for var in ("male", "education", "mother_education", "father_education", "smokenow", "ethnicity"):
    df[var] = df[var].astype('category')

In [ ]:
for var in (var for var in df.columns if var not in ("male", "education", "mother_education", "father_education", "smokenow", "ethnicity")):
    print(var)
    df[var] = df[var].astype(float)

In [ ]:
df.info()

---

# Dwa API

Implementacja biblioteki XGBoost w Pythonie dostarcza dwóch API:
1. scikit-learn API,
2. native API.

Scikit-learn API to wrapper native API, uwaga jednak na mniej lub bardziej oczywiste różnice w funkcjonalności pomiędzy API
(zob np. [Wątek na stackoverflow dot. różnic w API](https://stackoverflow.com/questions/37943403/difference-between-original-xgboost-learning-api-and-sklearn-xgbclassifier-sc)
lub
[dokumentację XGBoost dot. obsługi zmiennych kategorycznych](https://xgboost.readthedocs.io/en/stable/tutorials/categorical.html)).

# Zadanie 1

Utwórz prosty model XGBoost z użyciem scikit-learn API i dopasuj go do danych.
Pamiętaj, że zbiór danych zawiera zmienne kategoryczne - przygotuj odpowiednio dane i uwzględni to w modelu.

In [ ]:
X = ...
y = ...

In [ ]:
# Supported tree methods are `gpu_hist`, `approx`, and `hist`.
clf = ...

clf.fit(X, y)

In [ ]:
mean_squared_error(clf.predict(X), y)

7221799.253294746

# Zadanie 2

Utwórz prosty model XGBoost z użyciem native API. Pamiętaj o zmiennych kategorycznych.

In [ ]:
Xy = xgb.DMatrix(...)
booster = ...

In [ ]:
type(booster)

xgboost.core.Booster

In [ ]:
type(clf)

xgboost.sklearn.XGBRegressor

In [ ]:
mean_squared_error(booster.predict(Xy), y)

---

# Zadanie 3

Utwórz zbiór treningowy i testowy (20\% obserwacji do zbioru testowego). Użyj ziarna losowania równego `2023`.

Dopasuj model używając native API z następującymi ustawieniami:
* funkcja straty MSE,
* metryka ewaluacyjna RMSE, którą śledzi na zbiorze testowym,
* metoda budowania drzew `hist`,
* 400 boosterów.

Zapisz metryki na zbiorach train/test dla każdego z $m$ kroków budowania finalnego modelu.

In [ ]:
X_train, X_test, y_train, y_test = ...

dtrain = xgb.DMatrix(...)
dtest = xgb.DMatrix(...)

In [ ]:
squared_error = {
 ...
}

results = {}

booster = xgb.train(
...
)

Wykreśl metrykę RMSE na zbiorze treningowym i testowym w zależności od $m$, tj. ilości drzew w finalnym modelu.

In [ ]:
fig, ax = plt.subplots()

ax.plot(...)
ax.plot(...)

ax.legend(loc="best")

# Zadanie 4

Użyj scikit-learn API, aby dokonać kroswalidacji parametrów `eta` oraz `gamma`.

## Parametry

[opis parametrów booster'a z dok. XGBoost](https://xgboost.readthedocs.io/en/stable/parameter.html#parameters-for-tree-booster)

* $\eta$: "Step size shrinkage used in update to prevents overfitting. After each boosting step, we can directly get the weights of new features, and eta shrinks the feature weights to make the boosting process more conservative."
* $\gamma$: Minimum loss reduction required to make a further partition on a leaf node of the tree. The larger gamma is, the more conservative the algorithm will be.

Gamma is associated with "learning tree structure", while eta is associated with "model complexity" (it scales down gradients and thus speeds up/decreases pace of learning).

## `gamma`

[dok. XGBoost dot. parametru `gamma`](https://xgboost.readthedocs.io/en/stable/tutorials/model.html#learn-the-tree-structure):

\begin{equation}
Gain = \frac{1}{2} \left[\frac{G_L^2}{H_L+\lambda}+\frac{G_R^2}{H_R+\lambda}-\frac{(G_L+G_R)^2}{H_L+H_R+\lambda}\right] - \gamma
\end{equation}

Zob. także [artykuł na medium.com](
https://medium.com/data-design/xgboost-hi-im-gamma-what-can-i-do-for-you-and-the-tuning-of-regularization-a42ea17e6ab6).

## `eta`

Na podstawie [wpisu na medium.com](https://medium.com/syncedreview/tree-boosting-with-xgboost-why-does-xgboost-win-every-machine-learning-competition-ca8034c0b283)

\begin{align}
\hat{f}^{(m)}(x) = \hat{f}^{(m-1)}(x)  + &\hat{f}_m(x)
\\
&\hat{f}_m(x) = \eta \sum_{j=1}^T \hat{w}_{jm} I (x \in \hat{R}_{jm})
\end{align}

In [ ]:
gbm = xgb.XGBRegressor(
...
)

gbm_param_grid = {
...
}

grid_mse = GridSearchCV(
...
)

grid_mse.fit(X, y)

print("Best parameters found: ",grid_mse.best_params_)

Dopasowanie modelu z optymalnymi wartościami `eta` oraz `gamma`

In [ ]:
squared_error = {
...
}


booster_opt = xgb.train(
...
)

In [ ]:
fig, ax = plt.subplots()

...

ax.legend(loc="best")

# Zadanie 5

Co możemy zrobić, żeby stworzyć lepszy model?